In [1]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [5]:
test_df = pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)

In [3]:
def clear_word(w):
    x = w
    if x[:3] == 'is ' or x[:3] == 'to ':
        x = x[3:]
    if x[:3] == 'be ':
        x = x[3:]
    if x[len(x) - 1] == '.':\
        x = x[:len(x)-1]
    return x

In [6]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []
set_xi = set()
set_xe = set()
set_oe = set()

phrase_set = set([x for x in test_df.Event])

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set()
    for x in set(res_xi):
        if x != '':
            phrase2xintent[phrase].add(clear_word(x))
    res_xi.clear()
    
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set()
    for x in set(res_xe):
        if x != '':
            phrase2xemotion[phrase].add(clear_word(x))
    res_xe.clear()
    
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set()
    for x in set(res_oe):
        if x != '':
            phrase2oemotion[phrase].add(clear_word(x))
    res_oe.clear()

In [58]:
p_small = Predictor.from_path("ser_dir_original_model/model.tar.gz")
p_large = Predictor.from_path("ser_dir_large_model/model.tar.gz")
p_mid = Predictor.from_path("ser_dir_5e_model/model.tar.gz")
p_orig = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")

to new params <allennlp.common.params.Params object at 0x7f1ffd20f3c8>.


In [66]:
def intents(predictor, p):
    return set([" ".join(react) for react in predictor.predict(source=p)["xintent_top_k_predicted_tokens"]])
    
def xreact(predictor, p):
    return set([" ".join(react) for react in predictor.predict(source=p)["xreact_top_k_predicted_tokens"]])
    
def oreact(predictor, p):
    return set([" ".join(react) for react in predictor.predict(source=p)["oreact_top_k_predicted_tokens"]])

def predictor_info(predictor, p):
    print("Intent: ", intents(predictor, p))
    print("Xreact: ", xreact(predictor, p))
    print("Oreact: ", oreact(predictor, p))
    print()

def compare_orig_with(predictor, p):
    print("Intent: ", intents(p_orig, p)&intents(predictor, p))
    print("Xreact: ", xreact(p_orig, p)&xreact(predictor, p))
    print("Oreact: ", oreact(p_orig, p)&oreact(predictor, p))
    print()

def compare_orig_with_all(p):
    print("--------------------------------------------------------------")
    print("Фраза: ", p)
    print()
    
    print("Пересечение ответов оригинальной модели с натренированной на 10 эпохах")
    compare_orig_with(p_large, p)

    print("Пересечение ответов оригинальной модели с натренированной на 5 эпохах")
    compare_orig_with(p_mid, p)

    print("Пересечение ответов оригинальной модели с натренированной на 2 эпохах")
    compare_orig_with(p_small, p)

In [67]:
phrases = ["PersonX writes a code",
     "PersonX takes part in an intership",
     "PresonX codes all night long",
     "PersonX has tonns of deadlines",
     "PersonX suggested PersonY to work together"]

for p in phrases:
    compare_orig_with_all(p)

--------------------------------------------------------------
Фраза:  PersonX writes a code

Сравнение оригинальной модели с натренированной на 10 эпохах
Intent:  {'@@UNKNOWN@@', 'communicate', 'make a decision'}
Xreact:  {'satisfied', 'happy', 'excited'}
Oreact:  {'happy', 'none'}

Сравнение оригинальной модели с натренированной на 5 эпохах
Intent:  {'@@UNKNOWN@@', 'communicate', 'make a decision'}
Xreact:  {'satisfied', 'happy', 'helpful'}
Oreact:  {'happy', 'none'}

Сравнение оригинальной модели с натренированной на 2 эпохах
Intent:  {'@@UNKNOWN@@', 'none'}
Xreact:  {'happy', 'accomplished', 'satisfied', 'excited', 'helpful'}
Oreact:  {'happy', 'none'}

--------------------------------------------------------------
Фраза:  PersonX takes part in an intership

Сравнение оригинальной модели с натренированной на 10 эпохах
Intent:  {'responsible', 'none', 'helpful'}
Xreact:  {'satisfied', 'happy', 'excited'}
Oreact:  {'happy', 'thankful', 'none', 'relieved', 'grateful'}

Сравнение ориги